In [ ]:
from datetime import date
import random
import time
import yfinance as yf
import pandas as pd

import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from numpy.fft import fft, ifft, fftshift
import numpy as np
from numpy import log, sqrt, exp


from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.mixture import GaussianMixture


from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.stats.diagnostic import acorr_ljungbox

import scipy.stats as stats
from scipy.stats import probplot, laplace, norm, t, poisson
from scipy.linalg import solve_banded
from scipy.optimize import minimize, differential_evolution
from scipy.integrate import quad
from scipy.special import roots_laguerre
from scipy.interpolate import interp1d
from scipy.sparse import diags, kron, identity, csr_matrix
from scipy.sparse.linalg import spsolve
from scipy.stats import multivariate_normal, kstest

import statsmodels.api as sm
from statsmodels.nonparametric.kde import KDEUnivariate
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_process import ArmaProcess

import pymc as pm
import arviz as az

#import aesara.tensor as at

from tensorflow import keras
#from tensorflow.keras.utils import plot_model

#import pyswarms as ps

######################################
#from pmdarima import auto_arima
#from diptest import diptest



**##FACTOR MODELS**
Capital asset pricing model --> Arbitrage Pricing Theory (APT)
The Fama-French 3
% Factor models
Pastor and Stambaugh model

**Beyond MPT**
Mean variance --> skewness and kurtosis

**Risk types**

From varaince of return to -->

Shortfall probability - Roy's safety first criterion
Value at Risk
Conditional Value at risk or Expected shortfall
Worst case expectation

**Behavioural portfolio theory **
**Behavioural asset pricing model**


**Multi-period / Multi-scenario stochastic programming models**

In [ ]:
class PortfolioData:
  def __init__(self, rfr, dist, index_name):
    self.rfr = rfr
    self.dist = dist
    self.index_name = index_name
    self.pf = None
    self.index = None
    self.mu_index = None
    self.sigma_index = None
    self.mu_vector = None
    self.cov_matrix = None
    self.dof_t = None
    self.log_returns = None
    self.log_returns_index = None
    self.market_cap = None
    self.trading_days = 252

  def get_data_index(self):
    filename_1 = 'nifty50_database.db'
    ffilename_1 = '/content/drive/My Drive/LEARN/Finance/quantitative finance/Portfolio-optimization/nifty50_indices.xlsx'
    fffilename_1 = '/content/drive/My Drive/LEARN/Finance/quantitative finance/Portfolio-optimization/Nifty50_MarketCap.xlsx'
    pf_1 = (DownloadData(filename_1, ffilename_1, fffilename_1 ).query_all_stocks())
    index_1 = (DownloadData(ffilename_1, ffilename_1, fffilename_1).read_indices())
    market_cap_1 = (DownloadData(ffilename_1, ffilename_1, fffilename_1).read_market_cap())

    filename_2 = 'niftynext50_database.db'
    ffilename_2 = '/content/drive/My Drive/LEARN/Finance/quantitative finance/Portfolio-optimization/niftynext50_indices.xlsx'
    fffilename_2 = '/content/drive/My Drive/LEARN/Finance/quantitative finance/Portfolio-optimization/Nifty50_MarketCap.xlsx'
    pf_2 = (DownloadData(filename_2, ffilename_2, fffilename_2).query_all_stocks())
    index_2 = (DownloadData(ffilename_2, ffilename_2, fffilename_2).read_indices())

    if self.index_name == 'nifty50':
        self.pf = pf_1.copy()
        self.index = index_1[['Close']].copy()
        self.index.rename(columns={'Close': 'nifty50'}, inplace=True)
        self.market_cap = market_cap_1.copy()
    if self.index_name == 'niftynext50':
        self.pf = pf_2.copy()
        self.index = index_2[['Close']].copy()
        self.index.rename(columns={'Close': 'niftynext50'}, inplace=True)

    self.logreturn_index()

    return (pd.DataFrame({'Index': range(len(self.pf.columns)), 'Column Name': self.pf.columns}))
################################################################################################################################

  def logreturn_index(self):
      pfa = pf_analysis(self.index, self.dist)
      pfa.check_plot()
      self.log_returns_index = pfa.log_return_val()

      if self.dist == 'norm':
          self.mu_index, self.sigma_index = pfa.fit_marginal_univariate(self.log_returns_index)
      elif self.dist == 't':
          self.mu_index, self.sigma_index, self.dof_t = pfa.fit_marginal_univariate(self.log_returns_index)

  def logreturn_portfolio(self, col_name):
      pf = self.pf.iloc[:, col_name].copy()
      self.pf = pf.copy()

      pfa = pf_analysis(self.pf, self.dist)
      pfa.check_plot()
      pfa.CAGR()

      self.log_returns = pfa.log_return_val()

      if self.dist == 'norm':
          #self.mu_vector, self.cov_matrix = pfa.fit_marginal_multivariate(self.log_returns)
          self.mu_vector, self.cov_matrix = pfa.fit_joint(self.log_returns)
      elif self.dist == 't':
          #self.mu_vector, self.cov_matrix, self.dof_t = pfa.fit_marginal_multivariate(self.log_returns)
          self.mu_vector, self.cov_matrix, self.dof_t = pfa.fit_joint(self.log_returns)

  def Factor(self, opt_model):
    FM = FactorModels(self.rfr, self.log_returns, self.log_returns_index)
    if opt_model == 'Sharpe':
        result = FM.Sharpe('GLS') #option = OLS, GLS
    elif opt_model == 'CAPM':
        result = FM.CAPM('GLS') #option = OLS, GLS

    return result

################################################################################################################################
  def MPT(self, opt_optimize):
    ef = EfficientFrontier(self.rfr, self.mu_vector, self.cov_matrix, list(self.pf.columns))
    ef.plot_oppurtunity_set()  #n risky assets only
    ef.compute_frontier(opt_optimize) # minvar, maxret, cml

  def black_litterman(self, col, P, Q, C = None):

    ##################################################  PRIOR

    #market weights
    selected_caps = self.market_cap['MarketCap'].iloc[col].values
    total_market_cap = selected_caps.sum()
    market_weights = selected_caps / total_market_cap

    #covraince matrix of excess return
    rfr_daily = self.rfr / self.trading_days
    excess_pf = self.log_returns - rfr_daily
    cov_matrix = excess_pf.cov()
    cov_matrix *= self.trading_days

    cov_matrix_np = cov_matrix.to_numpy()
    mean_returns_np = self.log_returns.mean().to_numpy()
    mean_returns_np *= self.trading_days

    mu_market = float(market_weights @ mean_returns_np)
    sigma_market = np.sqrt(market_weights @ cov_matrix_np @ market_weights) #std of the return of market porfolio

    #Lambda - empirical
    lam_market = (mu_market - self.rfr) / sigma_market

    #Lambda - Litterman = w.T @ mu / sigma**2  = w.T @ mu /sigma [sharpe_ratio of market] / sigma [std of market]
    lam_market = 0.5 / sigma_market

    tau = self.trading_days / len(self.log_returns_index.dropna())


    print ("std of the excess market return", sigma_market)
    print ("risk aversion: ", lam_market)
    print ("Prior market weights", market_weights)
    print ("Prior expected excess market return", lam_market * (cov_matrix_np @ market_weights))
    print ("Prior variance excess market return", tau * cov_matrix_np)
    print ("tau: ", tau)

    ################################################### VIEWS / LIKELIHOOD
    P= np.array(P)
    Q=np.array(Q)

    ################################################### Run model
    bl = BlackLitterman(cov_matrix_np, market_weights, lam_market, tau)
    bl.set_views(P, Q, C)
    mu_bl, Sigma_bl, weights_bl = bl.run()

    print("Posterior excess Exp Returns:\n", mu_bl)
    print("Posterior excess Covariance Matrix of Returns:\n", Sigma_bl)
    print("Optimal Weights:\n", weights_bl)



